In [126]:
import pandas as pd
from codes.db import * 
from codes.lib import *
from codes.riverlog_for_gis import *
import pandasql as ps

gd={}
conn=connect_db()
if 1:
    load_ods(gd)
    riverlog_info_setup(gd)

sys 的 sheets:
 odict_keys(['table_def', 's_syspar', 's_info_point', 's_info_line', 's_info_area', 's_timeseq_rpt', 's_repeat_job', 's_topology_kind', 's_topology_transfer', 's_topology_node', 's_topology_edge']) 
basic 的 sheets:
 odict_keys(['table_def', 'b_表單說明', 'b_colmeta', 'b_水資源分區', 'b_水資源局', 'b_河川局', 'b_流域', 'b_河川', 'b_排水', 'b_水庫', 'b_水質水量保護區', 'b_水庫集水範圍', 'b_水庫集水區敏感區', 'b_堤防', 'b_排水設施', 'b_水門', 'b_抽水站', 'b_河川斷面樁', 'b_雨量站', 'b_河川水位站', 'b_浮標站', 'b_潮位站', 'b_地下水分區', 'b_地下水觀測井']) 
rain-station: output/rain-station.csv saved, shape = (2149, 6)
reservoir-info: output/reservoir-info.csv saved, shape = (152, 22)
waterLevel-station: output/waterLevel-station.csv saved, shape = (5176, 13)
waterLevelDrain-station: output/waterLevelDrain-station.csv saved, shape = (1189, 4)
waterLevelAgri-station: output/waterLevelAgri-station.csv saved, shape = (359, 4)
sewer-station: output/sewer-station.csv saved, shape = (310, 8)
tide-station: output/tide-station.csv saved, shape = (86, 4)
pump-station: 

# 使用範例

In [129]:
#gd['base']['拓墣-Node']
#gd['riverlog']["r_rain_station"]
if 0:
    df = gd['basic']['河川局']
    #query_str = """SELECT * FROM df_8410 where site_id like '%東區%'"""
    query_str = """SELECT * FROM df where rvb_name='第一河川局'"""
    q1_df= ps.sqldf(query_str, locals())
#q1_df


,stationID,city,lat,lon,name,town
0,C1I230,南投縣,23.9637,120.8369,九份二山,國姓鄉
1,466940,基隆市,25.1351,121.7323,基隆,仁愛區
2,466900,新北市,25.1667,121.4407,淡水,淡水區
3,466880,新北市,24.9994,121.4338,板橋,板橋區
4,466930,臺北市,25.1639,121.5363,陽明山,北投區
...,...,...,...,...,...,...
2144,81S450,臺東縣,23.0149,121.1285,加樂,海端鄉
2145,81S530,臺東縣,22.8935,121.0637,紅葉,延平鄉
2146,81F860,臺中市,24.2828,121.2577,松茂,和平區
2147,81I510,南投縣,23.9609,120.8378,九份二山(2),國姓鄉


# DB 存取

In [3]:
conn=connect_db()

if 0:
    sql = "select * from rivercode order by river_id"
    df = sql_to_df(conn,sql)

if 0:
    df = gd['basic']['河川局']
    df.to_csv('output/河川局.csv')    

if 1:
    sql="""
    CREATE TABLE "河川局" (
        a DECIMAL, 
        rvb_no DECIMAL, 
        rvb_name VARCHAR, 
        area BOOLEAN, 
        "水資源分區代號" BOOLEAN
    );

    """
    sql_exec(conn,sql)
if 0:
    sql = 'select * from 河川局'
    df = sql_to_df(conn,sql)
if 1:
    close_db(conn)
#df

Connecting to the PostgreSQL database...
Database connection closed.


# 維護

head -n 20 河川局.csv | csvsql -i postgresql --no-constraints --tables 河川局

\copy "S_topology_edge" FROM 'output/S_topology_edge.csv' WITH (FORMAT csv);

In [128]:
import subprocess
import os
def exec_print(cmd_str):
    print("executing : %s" %(cmd_str)) 
    p = subprocess.Popen(cmd_str, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    for line in p.stdout.readlines():
        print(line)
    retval = p.wait()    
    

def ods_to_scr(tables,sql_name,psql_name,sql_drop_name):
    loc="/Volumes/F2020/opt/anaconda3/envs/py37/bin/csvsql"
    sqls=[]
    sqls_drop=[]
    table_list=[]
    if 'table_def' in tables:
        def_df=tables['table_def']
        table_list = def_df[def_df['type']=='T']['table_name'].to_list()
        print("table_list = %s" %(table_list))
    for t_name in tables:
        if (not t_name=='table_def') and t_name in table_list:
            df = tables[t_name]
            sqls_drop.append("DROP TABLE IF EXISTS %s;" %(t_name))
            
            filename = "output/%s.csv" %(t_name)
            df.to_csv(filename,index=False)
            cmd_str="head -n 1000 output/%s.csv | %s -i postgresql --no-constraints --tables '%s' >> %s" %(t_name,loc,t_name,sql_name)
            exec_print(cmd_str)
            psql="\\copy \"%s\" FROM 'output/%s.csv' WITH (FORMAT csv,HEADER);" %(t_name,t_name)
            sqls.append(psql)
    
    with open(psql_name, 'a') as f:
        for line in sqls:
            f.write("%s\n" %(line))

    with open(sql_drop_name, 'a') as f:
        for line in sqls_drop:
            f.write("%s\n" %(line))

sql_drop_name="output/tb_drop.sql"
sql_name = "output/tb.sql"
psql_name = "output/tb.psql"

#產生 sql, psql
if 0:
    if os.path.exists(sql_drop_name):
        os.remove(sql_drop_name)
    if os.path.exists(sql_name):
        os.remove(sql_name)
    if os.path.exists(psql_name):
        os.remove(psql_name)

    #ods_to_scr(gd['sys'],sql_name,psql_name,sql_drop_name)
    ods_to_scr(gd['basic'],sql_name,psql_name,sql_drop_name)
    #ods_to_scr(gd['riverlog'],sql_name,psql_name,sql_drop_name)

#manual modify schema
# r_waterlevel_station,b_河川水位站
    
#drop, create table
if 1:
    conn=connect_db()
    with open (sql_drop_name, "r") as scrfile:
        sqls=scrfile.readlines()

    sql_str = "".join(sqls)
    print("sql_drop script:\n%s" % (sql_str))
    sql_exec(conn,sql_str)
    
    with open (sql_name, "r") as scrfile:
        sqls=scrfile.readlines()

    sql_str = "".join(sqls)
    print("sql script:\n%s" % (sql_str))
    sql_exec(conn,sql_str)

#import data
if 1:
    psql="/Applications/Postgres.app/Contents/Versions/13/bin/psql"
    with open (psql_name, "r") as sqlfile:
        sqls=sqlfile.readlines()
    psql_str = "".join(sqls)
    print("psql script:\n%s" %(psql_str))

    cmd_str="%s -h localhost -p 5431 -U postgres postgis -f %s -a" %(psql,psql_name)
    exec_print(cmd_str)



Connecting to the PostgreSQL database...
sql_drop script:
DROP TABLE IF EXISTS b_水資源分區;
DROP TABLE IF EXISTS b_水資源局;
DROP TABLE IF EXISTS b_河川局;
DROP TABLE IF EXISTS b_流域;
DROP TABLE IF EXISTS b_河川;
DROP TABLE IF EXISTS b_排水;
DROP TABLE IF EXISTS b_水庫;
DROP TABLE IF EXISTS b_水質水量保護區;
DROP TABLE IF EXISTS b_水庫集水範圍;
DROP TABLE IF EXISTS b_水庫集水區敏感區;
DROP TABLE IF EXISTS b_堤防;
DROP TABLE IF EXISTS b_排水設施;
DROP TABLE IF EXISTS b_水門;
DROP TABLE IF EXISTS b_抽水站;
DROP TABLE IF EXISTS b_河川斷面樁;
DROP TABLE IF EXISTS b_雨量站;
DROP TABLE IF EXISTS b_河川水位站;
DROP TABLE IF EXISTS b_浮標站;
DROP TABLE IF EXISTS b_潮位站;
DROP TABLE IF EXISTS b_地下水分區;
DROP TABLE IF EXISTS b_地下水觀測井;

sql script:
CREATE TABLE "b_水資源分區" (
	id VARCHAR, 
	name VARCHAR, 
	"area(sq_km)" DECIMAL, 
	x DECIMAL, 
	y DECIMAL
);
CREATE TABLE "b_水資源局" (
	wr_dist_no DECIMAL, 
	name VARCHAR
);
CREATE TABLE "b_河川局" (
	rvb_no DECIMAL, 
	rvb_name VARCHAR, 
	area BOOLEAN, 
	"水資源分區代號" BOOLEAN
);
CREATE TABLE "b_流域" (
	basin_id DECIMAL, 
	basin_cnam

psql script:
\copy "b_水資源分區" FROM 'output/b_水資源分區.csv' WITH (FORMAT csv,HEADER);
\copy "b_水資源局" FROM 'output/b_水資源局.csv' WITH (FORMAT csv,HEADER);
\copy "b_河川局" FROM 'output/b_河川局.csv' WITH (FORMAT csv,HEADER);
\copy "b_流域" FROM 'output/b_流域.csv' WITH (FORMAT csv,HEADER);
\copy "b_河川" FROM 'output/b_河川.csv' WITH (FORMAT csv,HEADER);
\copy "b_排水" FROM 'output/b_排水.csv' WITH (FORMAT csv,HEADER);
\copy "b_水庫" FROM 'output/b_水庫.csv' WITH (FORMAT csv,HEADER);
\copy "b_水質水量保護區" FROM 'output/b_水質水量保護區.csv' WITH (FORMAT csv,HEADER);
\copy "b_水庫集水範圍" FROM 'output/b_水庫集水範圍.csv' WITH (FORMAT csv,HEADER);
\copy "b_水庫集水區敏感區" FROM 'output/b_水庫集水區敏感區.csv' WITH (FORMAT csv,HEADER);
\copy "b_堤防" FROM 'output/b_堤防.csv' WITH (FORMAT csv,HEADER);
\copy "b_排水設施" FROM 'output/b_排水設施.csv' WITH (FORMAT csv,HEADER);
\copy "b_水門" FROM 'output/b_水門.csv' WITH (FORMAT csv,HEADER);
\copy "b_抽水站" FROM 'output/b_抽水站.csv' WITH (FORMAT csv,HEADER);
\copy "b_河川斷面樁" FROM 'output/b_河川斷面樁.csv' WITH (FORMAT csv,HEADER);
\co

# Debug

In [95]:
df=gd['riverlog']["r_waterlevel_station"]
#df[df['AlertLevel1'].notnull()]
f1=df['AlertLevel1'].notnull()
#df[f1 & df['AlertLevel1']=='{}']
df2=df[f1 & df['AlertLevel1'].str.contains('\{\}',regex=True)]
#df[df['BasinIdentifier']=='1140H002']
df2

#df.dtypes
#df.to_csv('output/r_waterlevel_station.csv')

,BasinIdentifier,AffiliatedBasin,AffiliatedSubSubsidiaryBasin,AffiliatedSubsidiaryBasin,AlertLevel1,AlertLevel2,AlertLevel3,LocationAddress,ObservatoryName,RiverName,TownIdentifier,lat,lon
